# Perceptron multicapa: Clasificación
En este notebook trabajaremos sobre predicción de Churn. Específicamente, decidir si una persona dejará un banco o no dadas sus características usando una red neuronal fully connected.

In [13]:
# importar librerias necesarias
import numpy as np
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader

## Parte 1. Manejo de datos
### 1. Importar el conjunto de datos / EDA

In [14]:
dataset = pd.read_csv('Churn_Modelling.csv')
print(dataset.columns)
print(dataset.info())
print(dataset.head())

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  1

### 2. Procesamiento de datos
Primero removemos la variable dependiente `Exited` ya que esta representa la etiqueta. De esta manera separamos nuestros datos en atributos `X` y etiqueta `y`.

In [15]:
feature_cols = ['CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary']
X = dataset[feature_cols]
y = dataset['Exited']
print(X.shape, y.shape, y[:10])

(10000, 10) (10000,) 0    1
1    0
2    1
3    0
4    0
5    1
6    0
7    1
8    0
9    0
Name: Exited, dtype: int64


### Ingeniería de atributos

Tenemos diferentes tipos de variables, algunas numéricas y otras de tipo categórico o `object` en pandas. Podemos usar diferentes tipos de codificadores para los categóricos:
- [OneHotEncoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html#sklearn.preprocessing.OneHotEncoder)
- [OrdinalEncoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OrdinalEncoder.html#sklearn.preprocessing.OrdinalEncoder)

Otras transformaciones comunes son la normalización:
- [StandardScaler](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html)

In [16]:
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler


# Diccionario de encoders feature: encoder para poder procesar los datos
encoders = [
    # handle_unknown='ignore' para evitar errores si hay categorias nuevas en test set
    # unknown sera codificado como otra categoria
    ("gender_enc", OneHotEncoder(handle_unknown='ignore'), ["Gender"]),
    ("geo_enc", OrdinalEncoder(), ["Geography"]),
]

# Solo aplicamos los encoders a las columnas categóricas
# las demas columnas se dejan igual (remainder='passthrough')
preprocessor = ColumnTransformer(encoders, remainder='passthrough')
X_processed = preprocessor.fit_transform(X)

# Nota como X_processed es un numpy array, ya no tenemos los nombres de las columnas y no tenemos un pd dataframe
print(X_processed[:5])

[[1.0000000e+00 0.0000000e+00 0.0000000e+00 6.1900000e+02 4.2000000e+01
  2.0000000e+00 0.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0134888e+05]
 [1.0000000e+00 0.0000000e+00 2.0000000e+00 6.0800000e+02 4.1000000e+01
  1.0000000e+00 8.3807860e+04 1.0000000e+00 0.0000000e+00 1.0000000e+00
  1.1254258e+05]
 [1.0000000e+00 0.0000000e+00 0.0000000e+00 5.0200000e+02 4.2000000e+01
  8.0000000e+00 1.5966080e+05 3.0000000e+00 1.0000000e+00 0.0000000e+00
  1.1393157e+05]
 [1.0000000e+00 0.0000000e+00 0.0000000e+00 6.9900000e+02 3.9000000e+01
  1.0000000e+00 0.0000000e+00 2.0000000e+00 0.0000000e+00 0.0000000e+00
  9.3826630e+04]
 [1.0000000e+00 0.0000000e+00 2.0000000e+00 8.5000000e+02 4.3000000e+01
  2.0000000e+00 1.2551082e+05 1.0000000e+00 1.0000000e+00 1.0000000e+00
  7.9084100e+04]]


In [17]:
# Adicionalmente, normalizamos los datos numéricos
sc = StandardScaler()
X_processed = sc.fit_transform(X_processed)

print("Datos procesados y normalizados:")
print(X_processed[:5])

Datos procesados y normalizados:
[[ 1.09598752 -1.09598752 -0.90188624 -0.32622142  0.29351742 -1.04175968
  -1.22584767 -0.91158349  0.64609167  0.97024255  0.02188649]
 [ 1.09598752 -1.09598752  1.51506738 -0.44003595  0.19816383 -1.38753759
   0.11735002 -0.91158349 -1.54776799  0.97024255  0.21653375]
 [ 1.09598752 -1.09598752 -0.90188624 -1.53679418  0.29351742  1.03290776
   1.33305335  2.52705662  0.64609167 -1.03067011  0.2406869 ]
 [ 1.09598752 -1.09598752 -0.90188624  0.50152063  0.00745665 -1.38753759
  -1.22584767  0.80773656 -1.54776799 -1.03067011 -0.10891792]
 [ 1.09598752 -1.09598752  1.51506738  2.06388377  0.38887101 -1.04175968
   0.7857279  -0.91158349  0.64609167  0.97024255 -0.36527578]]


### 3. Uniendo todo el manejo de datos...

Para entrenar siempre tenemos que tener un split de train y validación. 
Vamos a primero generar el split y después aplicar el procesamiento de la celda anterior para evitar el data leakage.
En la siguiente celda se resume el codigo anterior

In [18]:
from sklearn.model_selection import train_test_split

# leer datos
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset[feature_cols]
y = dataset['Exited']

# Separar en train y test, 20% de los datos van a validacion
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Transformar a numpy arrays
y_train = y_train.values
y_test = y_test.values

# Preprocesamiento de datos
def preprocess(X, train=False, encoders=None, scaler=None):
    if train:
        encoders = [
            ("gender_enc", OneHotEncoder(handle_unknown='ignore'), ["Gender"]),
            ("geo_enc", OneHotEncoder(handle_unknown='ignore'), ["Geography"]),
        ]
        preprocessor = ColumnTransformer(encoders, remainder='passthrough')
        X_encoded = preprocessor.fit_transform(X)
        print(X_encoded.shape)
        scaler = StandardScaler()
        X_processed = scaler.fit_transform(X_encoded)

        return X_processed, preprocessor, scaler
    else:
        X_encoded = encoders.transform(X)  # encoders = preprocessor ya entrenado
        X_processed = scaler.transform(X_encoded)
        return X_processed

X_train_processed, preprocessor, scaler = preprocess(X_train, train=True)

# Aplicar procesamiento de datos a validacion, ya no se usa fit_transform sino solo transform
# le mandamos los encoders y el normalizador entrenado
X_test_processed = preprocess(X_test, train=False, encoders=preprocessor, scaler=scaler)



(8000, 13)


Como visto en clase, los codificadores le asignan un valor numerico a un categórico. Para poder saber a que se asignó cada categórico corre la siguiente celda. También podras visualizar la información que guarda el normalizador

In [19]:
def viz_encoders(preprocessor, scaler):
    print("Encoders usados:")
    for name, encoder, cols in preprocessor.transformers_:
        if name != 'remainder':
            print(f"\n- {name}: {encoder} en columnas {cols}")
            if hasattr(encoder, 'categories_'):  # OrdinalEncoder
                print(f"  Categorías: {encoder.categories_}")
            if hasattr(encoder, 'get_feature_names_out'):  # OneHotEncoder
                print(f"  Columnas codificadas: {encoder.get_feature_names_out(cols)}")
    
    print("\nScaler usado:")
    print(scaler)
    
    # Imprimir medias y desviaciones del StandardScaler
    if hasattr(scaler, 'mean_') and hasattr(scaler, 'scale_'):
        print("\nMedias del StandardScaler:")
        print(scaler.mean_)
        print("Desviaciones estándar del StandardScaler:")
        print(scaler.scale_)

viz_encoders(preprocessor, scaler)

def show_column_mapping(preprocessor):
    col_index = 0
    for name, transformer, cols in preprocessor.transformers_:
        if name != 'remainder':
            if hasattr(transformer, 'get_feature_names_out'):  # OneHot
                out_cols = transformer.get_feature_names_out(cols)
            else:  # Ordinal o 1 columna
                out_cols = cols
            for c in out_cols:
                print(f"Columna {col_index}: {c}")
                col_index += 1

    # Columns passthrough
    if preprocessor.remainder == 'passthrough':
        passthrough_cols = preprocessor.transformers_[-1][-1]  # lista de columnas pasadas
        # O si no se listan, agregarlas manualmente
        print("Las columnas restantes (passthrough) se añaden al final")

show_column_mapping(preprocessor)

print(X_train_processed[:1])

Encoders usados:

- gender_enc: OneHotEncoder(handle_unknown='ignore') en columnas ['Gender']
  Categorías: [array(['Female', 'Male'], dtype=object)]
  Columnas codificadas: ['Gender_Female' 'Gender_Male']

- geo_enc: OneHotEncoder(handle_unknown='ignore') en columnas ['Geography']
  Categorías: [array(['France', 'Germany', 'Spain'], dtype=object)]
  Columnas codificadas: ['Geography_France' 'Geography_Germany' 'Geography_Spain']

Scaler usado:
StandardScaler()

Medias del StandardScaler:
[4.56250000e-01 5.43750000e-01 5.07250000e-01 2.45125000e-01
 2.47625000e-01 6.50550000e+02 3.88546250e+01 4.98075000e+00
 7.60765042e+04 1.53237500e+00 7.07750000e-01 5.15875000e-01
 1.00172283e+05]
Desviaciones estándar del StandardScaler:
[4.98082260e-01 4.98082260e-01 4.99947435e-01 4.30161289e-01
 4.31632783e-01 9.70033556e+01 1.04488631e+01 2.88996184e+00
 6.25774532e+04 5.77669334e-01 4.54796589e-01 4.99747921e-01
 5.75348298e+04]
Columna 0: Gender_Female
Columna 1: Gender_Male
Columna 2: Geogr

## Parte 2. Construyendo el modelo

### Crear la red neuronal

La red neuronal fully connected en pytorch se implementa de forma muy sencilla. Se escpefician los tamaños de las matrices de peso y la función de activación.

In [20]:
model = nn.Sequential(
    nn.Linear(X_train_processed.shape[1], 1024),
    nn.ReLU(),
    nn.Linear(1024, 512),
    nn.ReLU(),
    nn.Linear(512, 128),
    nn.ReLU(),
    nn.Linear(128, 1),
    nn.Sigmoid()
)


## Part 3. Entrenamiento

Las redes neuronales se entrenan con descenso de gradiente. En la siguiente sección implementamos descenso de gradiente para aplicar a la red anterior en el conjunto de datos. Para esto se tiene que definir 3 elementos principales:
1. El DataLoader. Este especifica como se van a ir cargando los datos y automatiza el proceso de separlo en minibatches
2. El optimizador. este especifica la regla de actualización de los pesos en su implementación (Por eso aquí se especifica la tasa de aprendizaje)
3. La función de costo. Esta especifica el costo del cual se calculará el gradiente. El optimizador utiliza esta función para guiar la actualización de los pesos. Tenemos que escoger una función apropiada al problema que queremos resolver, en este caso como es clasificación binaria se utiliza el BCELoss.

In [21]:
optimizer = optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.BCELoss()

print(X_train_processed.shape, y_train.shape)

train_data = []
for i in range(X_train_processed.shape[0]):
   x_i = X_train_processed[i].astype(np.float32) # 11,

   y_i = y_train[i].astype(np.float32)  # scalar
   y_i = np.expand_dims(y_i, axis=-1)  # 1,

   train_data.append([x_i, y_i])
train_loader = DataLoader(train_data, batch_size=7, shuffle=True)

(8000, 13) (8000,)


### Descenso de gradeinte
Una vez definido lo anterior simplemente realizamos descenso de gradiente sobre la red

In [22]:
epochs = 10

for epoch in range(epochs):
	losses = []
	for x_batch, y_batch in train_loader:
		optimizer.zero_grad()

		# Forward pass
		predictions = model(x_batch)

		# Calculo del loss
		loss = loss_fn(predictions, y_batch)

		# Calculo del gradiente
		loss.backward()

		# Actualizacion de pesos w = w - lr * grad
		optimizer.step()

		# Agregar loss de este batch a la lista para promediar
		losses.append(loss.item())

		# Agregar un step de valoración si se desea
		# val_step(model, X_val, y_val)
	print(f"epoch {epoch}: training loss {np.mean(losses)}")

epoch 0: training loss 0.395110987027421
epoch 1: training loss 0.3567352691536553
epoch 2: training loss 0.3461608087948718
epoch 3: training loss 0.3405612688609387
epoch 4: training loss 0.3374840562099025
epoch 5: training loss 0.33166928253970124
epoch 6: training loss 0.33224859811880564
epoch 7: training loss 0.3269090829897845
epoch 8: training loss 0.32175128092363553
epoch 9: training loss 0.3198302005200092
epoch 10: training loss 0.3146617976135443
epoch 11: training loss 0.30973480072374393
epoch 12: training loss 0.3068600448495395
epoch 13: training loss 0.3015042412969058
epoch 14: training loss 0.29445923983711386
epoch 15: training loss 0.28952852824551306
epoch 16: training loss 0.28446335412524415
epoch 17: training loss 0.27252235796550256
epoch 18: training loss 0.2696747699569526
epoch 19: training loss 0.2628941094048145


## Parte 4. Inferencia

Ahora que vimos como entrenar el modelo intenta predecir para un solo cliente de "prueba". Utiliza la red neuronal para decidir si el siguiente cliente dejará el banco.
- Geography: France
- Credit Score: 600
- Gender: Male
- Age: 40 years old
- Tenure: 3 years
- Balance: \$ 60000
- Number of Products: 2
- Does this customer have a credit card? Yes
- Is this customer an Active Member: Yes
- Estimated Salary: \$ 50000


**Solución**

In [23]:
print(feature_cols)
show_column_mapping(preprocessor)

['CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary']
Columna 0: Gender_Female
Columna 1: Gender_Male
Columna 2: Geography_France
Columna 3: Geography_Germany
Columna 4: Geography_Spain
Las columnas restantes (passthrough) se añaden al final


In [ ]:
# Creamos un dato de prueba
# Las primeras dos son el genero (one hot) y la tercera el pais (ordinal), las demas en el orden original
arr = np.array([[0, 1, 1, 0, 0, 600, 40, 3, 60000, 2, 1, 1, 50000]])

# Scaler es el ultimo que se entreno
processed = scaler.transform(arr)
inp_tensor = torch.from_numpy(processed).float()

# Hacemos la predicción
pred = model(inp_tensor)

# impromimir si es mayor a 0.5 (clase positiva)
print(model(inp_tensor) > 0.5)

tensor([[False]])


Por lo tanto, ¡nuestro modelo de redes neuronales predice que este cliente se queda en el banco! (false significa no hace churn)

**Nota importante 1**: Observa que los valores de las características se ingresaron todos dentro de un doble par de corchetes. Esto se debe a que el método predict siempre espera un array 2D como formato de entrada. Y al poner nuestros valores dentro de un doble par de corchetes, hacemos que la entrada sea exactamente un array 2D.

**Nota importante 2**: Observa también que el país "France" no se ingresó como una cadena en la última columna, sino como "1, 0, 0" en las tres primeras columnas. Esto se debe a que, por supuesto, el método predict espera los valores one-hot encoded del estado, y como vemos en la primera fila de la matriz de características X, "France" se codificó como "1, 0, 0". Y ten cuidado de incluir estos valores en las primeras tres columnas, porque las variables dummy siempre se crean en las primeras columnas.

### Prediciendo resultados de validación
Lo comun durante entrenamiento es en cada epoch evaluar TODO el conjunto de validación. En la celda de abajo se muestra una evaluación.

Normalmente esto estaría implementado dentro de un método y estaría dentro del for loop de descenso de gradiente

In [ ]:
X_test_torch = torch.from_numpy(X_test_processed).float()
y_test_torch = torch.from_numpy(y_test).float()

y_pred = model(X_test_torch)
y_pred = (y_pred > 0.5).numpy()

print(y_pred.shape, y_test.shape)

# Resultados, accoracy
y_test_reshaped = y_test.reshape(-1, 1)
print(y_test_reshaped.shape)
accuracy = np.mean(y_pred == y_test_reshaped)
print(f"Accuracy de validación: {accuracy * 100:.2f}%")
### Making the Confusion Matrix

Otras métricas que podemos calcular son la matriz de confusión o usar las funciones de sklearn para las métricas en lugar de implementarlas directamente

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)